## Notebook Setup

In order to be able to run the code, we need to install the *eagerx_tutorials* package and ROS.

### Activate GPU (**Colab only**)

When in Colab, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

In [1]:
try:
    import eagerx_tutorials
except ImportError:
    !{"echo 'Installing eagerx-tutorials with pip.' && pip install eagerx-tutorials >> /tmp/eagerx_install.txt 2>&1"}
if 'google.colab' in str(get_ipython()):
    !{"curl 'https://raw.githubusercontent.com/eager-dev/eagerx_tutorials/master/scripts/setup_colab.sh' > ~/setup_colab.sh"}
    !{"bash ~/setup_colab.sh"}

# Setup interactive notebook
# Required in interactive notebooks only.
from eagerx_tutorials import helper
helper.setup_notebook()

Not running on CoLab.
Execute ROS commands as "!...".
ROS melodic available.


# EAGERx Advanced usage - ICRA 2022

Github repo: https://github.com/araffin/tools-for-robotic-rl-icra2022

EAGERx: https://github.com/eager-dev/eagerx

Documentation: https://eagerx.readthedocs.io/en/master/


## Introduction

In this notebook, you will learn to use eagerx to create a gym-compatible environment. This tutorial covers:
- how to initialize a robot (Go 1 Quadruped Robot).
- how to add pre-processing nodes (i.e. low-level controllers).
- how to fine-tune low-level controllres to achieve the desired behavior.
- how to (de)select various sensors to investigate its effect on the learning performance.

In the remainder of this tutorial, we will go more into detail on these concepts.

Furthermore, you will be asked to add/modify a couple of lines of code, which are marked by

```python

# YOUR CODE HERE

# END OF YOUR CODE
```

## Quadruped Locomotion

In this tutorial, we will learn small offsets to the trot of a [Go 1 Quadruped](https://tribotix.com/product/go1-quadruped-robot/) so that the quadruped makes a left-turn instead of going straight ahead. For this, we will start with an open-loop trot gait that uses a central pattern generator ([CPG](https://en.wikipedia.org/wiki/Central_pattern_generator#Locomotion)) to walk straight ahead. Central pattern generators have been used to control the motion of individual joints of walking robots and in this tutorial we will use one that is based on the Hopf oscillator.

In a nutshell, we will use a Hopf network to produce desired cartesian coordinates for the quadruped's feet. Then, the cartesian coordinates are mapped to desired joint positions using the forward kinematics of the quadruped. The desired joint positions are then tracked using the quadruped's joint controllers.

## Let's get started

First we will import EAGERx and initialize it.
EAGERx makes use of ROS functionality for communication and during initialization a ROS master is started if there isn't one running already.

In [2]:
import eagerx
# Initialize eagerx (starts roscore if not already started.)
eagerx.initialize("eagerx_core", log_level=eagerx.log.INFO)

... logging to /home/bas/.ros/log/5a5a9c74-d95e-11ec-9269-02421573ea46/roslaunch-bas-Alienware-m15-R2-24810.log
started roslaunch server http://172.18.0.1:46771/
ros_comm version 1.14.11


SUMMARY

PARAMETERS
 * /rosdistro: melodic
 * /rosversion: 1.14.11

NODES

auto-starting new master
process[master]: started with pid [24863]
ROS_MASTER_URI=http://localhost:11311
setting /run_id to 5a5a9c74-d95e-11ec-9269-02421573ea46
process[rosout-1]: started with pid [24885]
started core service [/rosout]


## Graph definition
Next, we initialize a [Go 1 Quadruped](https://tribotix.com/product/go1-quadruped-robot/) that we prepared for this tutorial.

It has various sensors available that run at 20 Hz:
- `joint_position`: the position of the quadruped's leg joints in radians, so 4 legs with 2 joints each.
- `joint_velocity`: the angular velocity in radians of the quadruped's leg joints.
- `force_torque`: The joint reaction forces of the quadruped's leg joints (Fx, Fy, Fz, Mx, My, Mz).
- `orientation`: the orientation of the quadruped's body in quaternions (x, y, z, w).
- `position`: the global cartesian position of the quadruped's body in meter (x, y, z).
- `velocity`: the global cartesian velocity of the quadruped's body in meters per second (dx, dy, dz).

and an actuator that runs at 200 Hz:
- `joint_control`: Tracks the desired joint positions of the quadruped's leg joints in radians.

In [3]:
# Available sensors
sensors = ["joint_position", "joint_velocity", "force_torque", "orientation", "position", "velocity"]
actuators = ["joint_control"]

# Create the GO 1 quadruped
import eagerx_tutorials.quadruped.object
robot = eagerx.Object.make("Quadruped", "quadruped", actuators=actuators, sensors=sensors, rate=20)

# Set the quadruped's control rate to 200 Hz.
robot.actuators.joint_control.rate = 200

pybullet build time: Apr 26 2022 03:12:14


Then, we initialize the central pattern generator and cartesian control nodes.

In [4]:
# Create central pattern generator (uses Hopf Oscillator)
import eagerx_tutorials.quadruped.cpg_gait
cpg = eagerx.Node.make("CpgGait", "cpg", rate=200, gait="TROT", omega_swing=16 * 3.14, omega_stance=4 * 3.14)

# Create cartesian control (uses the quadruped's forward kinematics)
import eagerx_tutorials.quadruped.cartesian_control
cartesian_control = eagerx.Node.make("CartesiandPDController", "cartesian_control", rate=200)

We add the robot and the two nodes to an empty graph.

Then, we connect the output of the central pattern generator (i.e. cartesian feet positions) to the cartesian controller, that in turn, maps them to desired joint positions that are passed to the quadruped's joint controller.

In [5]:
# Initialize empty graph
graph = eagerx.Graph.create([robot, cartesian_control, cpg])

# Interconnect the nodes that results in an initial trot (that moves straight ahead).
graph.connect(source=cpg.outputs.cartesian_pos, target=cartesian_control.inputs.cartesian_pos)
graph.connect(source=cartesian_control.outputs.joint_pos, target=robot.actuators.joint_control)

This open-loop control strategy produces in the nominal case a **forward trot** as demonstrated by the two quadrupeds below:

<img src="./figures/quad.gif"/>

However, we want the quadruped to make a **left-turn** instead!


Therefore, we will define an environment action called `offset`. This offset will be added to the generated open-loop feet pattern.

The offset action range (i.e. high & low) are chose to be relatively small (~1cm). Nonetheless, this should be enough to modify the pattern such that it makes the quadruped turn.


In [6]:
graph.connect(action="offset", target=cpg.inputs.offset)

Then, we connect the sensor as observations in the graph that we would like to use

In [7]:
# Select the sensors that are to be used as observations
# graph.connect(observation="joint_position", source=robot.sensors.joint_position)
# graph.connect(observation="joint_velocity", source=robot.sensors.joint_velocity)
# graph.connect(observation="position", source=robot.sensors.position)
graph.connect(observation="force_torque", source=robot.sensors.force_torque)
# graph.connect(observation="velocity", source=robot.sensors.velocity)
graph.connect(observation="orientation", source=robot.sensors.orientation, window=2)

# The open-loop pattern is probably also informative to determine relevant offsets.
initial_obs = [-0.01354526, -0.26941818, 0.0552178, -0.25434446]
graph.connect(observation="xs_zs", source=cpg.outputs.xs_zs, skip=True, initial_obs=initial_obs)

We will also visualize the quadruped's movement during training. To reduce the computational overhead of rendering 3D images on the training speed, we will render the quadruped's (x,y) cartesian coordinates as a proxy instead.

For this, we have already prepared the `XyPlane` node.

In [10]:
# Create xy-plane render node
import eagerx_tutorials.quadruped.overlay # Registers the xy plane node
xy_plane = eagerx.Node.make("XyPlane", "xy_plane", rate=5)

# Add node to graph
graph.add(xy_plane)

# The node renders images, based on the x,y position sensor measurements.
graph.connect(source=robot.sensors.position, target=xy_plane.inputs.position)

# Select the output of the node for rendering.
graph.render(xy_plane.outputs.image, rate=5)

It is also possible to inspect the graph using the `eagerx-gui` package.

It can be installed as follows:
```bash
pip3 install eagerx-gui
```
Jupyter notebooks have limited support for interactive applications, so we cannot open the GUI here.
But if we were to run
```python
graph.gui()
```
If we select all the quadruped's sensors as observations, the gui would show the following:

<img src="./figures/tutorial_2_gui.svg" width=720>

Here we see that the actions of the agent are outputs of *env/actions* and that the observations of the agent are inputs of *env/observations*.
Also, the image output connected to *env/render* will be rendered.
Note that *env/actions*, *env/observations* and *env/render* are connections of the `Graph` to the environment.
They are split up in the GUI as nodes for visualization purposes.

## Environment definition
Then, we define the environment that takes the graph and pybullet engine as input. The actions and observations of the environment are the ones we previously registered in the graph.


### **Exercise 1**
In this exercise, we are going to finish `QuadrupedEnv.step(action)` Specifically, we need to define a reward function that promotes offsets that make the quadruped's trot turn left. We will do so by specifying a reward function:
- Estimate the current `yaw_rate` using the last two yaw sensor measurements. *hint: don't forget to multiply the difference with the sensor rate (`=20 Hz`)*.
- Calculate the `desired_yaw_rate` (e.g. ~20 degrees) and convert it to radians. *hint: use `np.deg2rad(deg)` to convert degrees to radians.*
- Define reward function as the negative squared error between the desired and actual yaw rate.
- Add a little positive *alive* bonus (e.g. 0.25) to the reward, so that the quadruped does not fall down.


In [13]:
# Define Gym Environment
import pybullet
import numpy as np
import gym
from typing import Dict, Tuple

class QuadrupedEnv(eagerx.BaseEnv):
    def __init__(self, name, rate, graph, engine, episode_timeout):
        super(QuadrupedEnv, self).__init__(name, rate, graph, engine, force_start=True)
        self.steps = None
        self.timeout_steps = int(episode_timeout * rate)
        self.rate = rate  # [Hz] Sensor rate

    @property
    def observation_space(self) -> gym.spaces.Dict:
        """The Space object corresponding to valid observations.

        Per default, the observation space of all registered observations in the graph is used.
        """
        return self._observation_space

    @property
    def action_space(self) -> gym.spaces.Dict:
        """The Space object corresponding to valid actions

        Per default, the action space of all registered actions in the graph is used.
        """
        return self._action_space

    def reset(self):
        """A method that resets the environment to an initial state and returns an initial observation."""
        # Reset number of steps
        self.steps = 0

        # Sample desired states
        states = self.state_space.sample()

        # Perform reset
        obs = self._reset(states)
        return obs

    def step(self, action: Dict) -> Tuple[Dict, float, bool, Dict]:
        """A method that runs one timestep of the environment's dynamics."""
        
        # Here, we apply a step (i.e. we step the graph dynamics).
        # It returns a dict containing measurements of all registered observations.
        obs = self._step(action)
        self.steps += 1

        # We have access to the last two orientation sensor measurements, 
        # because we used window=2 when connecting `orientation` as an observation in the graph.
        _, _, prev_yaw = pybullet.getEulerFromQuaternion(obs["orientation"][-2])
        roll, pitch, yaw = pybullet.getEulerFromQuaternion(obs["orientation"][-1])
        
        # YOUR CODE HERE
        # 1. Calculate the yaw rate using prev_yaw and yaw (don't forget to scale with self.rate).
        # 2. Calculate the desired yaw_rate (20 degrees) in radians.
        # 3. Calculate the negative squared error between the desired and actual yaw rate.
        # 4. Add a little alive bonus to promote not falling down.
        yaw_rate = (yaw - prev_yaw) * self.rate
        desired_yaw_rate = np.deg2rad(20) # [rad] Desired yaw rate
        yaw_cost = (yaw_rate - desired_yaw_rate) ** 2
        alive_bonus = 0.25
        reward = alive_bonus - yaw_cost
        # END OF YOUR CODE

        # Determine termination condition
        has_fallen = abs(np.rad2deg(roll)) > 40 or abs(np.rad2deg(pitch)) > 40
        timeout = self.steps >= self.timeout_steps

        # Determine done flag
        done = timeout or has_fallen
        
        # Set info about episode truncation
        info = {"TimeLimit.truncated": timeout and not has_fallen}
        return obs, reward, done, info

# Define the pybullet engine
import eagerx_pybullet.engine  # Registers PybulletEngine
engine = eagerx.Engine.make("PybulletEngine", rate=200, gui=False, egl=False, process=eagerx.process.ENVIRONMENT)    

# Initialize Environment
episode_timeout = 10  # [s] number of seconds before timing-out an episode.
env = QuadrupedEnv(name="QuadEnv", rate=20, graph=graph, engine=engine, episode_timeout=episode_timeout)

[WARN] [1653176014.827049]: Adding state "quadruped/image/pos" to simulation node "quadruped/image" can potentially make the agnostic environment with object "quadruped" engine-specific. Check the spec of "Quadruped" under engine implementation "PybulletEngine" for more info.
[WARN] [1653176014.828500]: Adding state "quadruped/image/orientation" to simulation node "quadruped/image" can potentially make the agnostic environment with object "quadruped" engine-specific. Check the spec of "Quadruped" under engine implementation "PybulletEngine" for more info.


[INFO] [1653176014.889730]: Node "/QuadEnv/env/supervisor" initialized.
[INFO] [1653176015.048248]: Node "/QuadEnv/engine" initialized.
[INFO] [1653176015.332893]: Node "/QuadEnv/environment" initialized.
[INFO] [1653176015.487639]: Node "/QuadEnv/cartesian_control" initialized.
TROT
[INFO] [1653176015.554319]: Node "/QuadEnv/cpg" initialized.
[INFO] [1653176015.556167]: Waiting for nodes "['env/render']" to be initialized.
[INFO] [1653176015.591802]: Node "/QuadEnv/xy_plane" initialized.
[INFO] [1653176015.760613]: Adding object "quadruped" of type "Quadruped" to the simulator.
/home/bas/eagerx_dev/eagerx_tutorials/eagerx_tutorials/quadruped/go1/go1_description/urdf/go1.urdf
[INFO] [1653176016.360124]: Node "/QuadEnv/quadruped/joint_position" initialized.
[INFO] [1653176016.376369]: Node "/QuadEnv/quadruped/joint_velocity" initialized.
[INFO] [1653176016.393423]: Node "/QuadEnv/quadruped/force_torque" initialized.
[INFO] [1653176016.409283]: Node "/QuadEnv/quadruped/orientation" initi

## Training

<img src="./figures/train_eps_3.gif"/>

In [15]:
# Stable-baselines
from sb3_contrib import TQC
from eagerx.wrappers import Flatten

# Define hyper parameters for the TQC policy.
hyperparams = dict(
    learning_rate=1e-3,
    tau=0.02,
    gamma=0.98,
    buffer_size=300000,
    learning_starts=100,
    use_sde=True,
    use_sde_at_warmup=True,
    train_freq=8,
    gradient_steps=10,
    verbose=1,
    top_quantiles_to_drop_per_net=0,
    policy_kwargs=dict(n_critics=1, net_arch=dict(pi=[64, 64], qf=[64, 64])),
)

# Initialize the model
model = TQC("MlpPolicy", Flatten(env), **hyperparams)

# Train for 30 episodes
train_episodes = 30
try:
    train_steps = int(train_episodes * episode_timeout * 20)
    # Render top-view of the quadruped's movement
    env.render("human")
    # Start training!
    model.learn(train_steps)
    # Save the final policy
    model.save("last_policy")
except KeyboardInterrupt:
    model.save("last_policy")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
[INFO] [1653176031.149196]: Nodes initialized.
[INFO] [1653176031.423836]: Pipelines initialized.
argv[0]=
argv[0]=
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | 4.39     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 16       |
|    time_elapsed    | 48       |
|    total_timesteps | 800      |
| train/             |          |
|    actor_loss      | -21.3    |
|    critic_loss     | 0.397    |
|    ent_coef        | 0.402    |
|    ent_coef_loss   | -5.97    |
|    learning_rate   | 0.001    |
|    n_updates       | 870      |
|    std             | 0.05     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | 7.3      |
| time/              |          |
|    ep

## Evaluation

<img src="./figures/eval_eps_4.gif"/>


In [ ]:
# Evaluate
from eagerx_tutorials.quadruped.evaluate import EvaluateEnv
from stable_baselines3.common.evaluation import evaluate_policy

engine = eagerx.Engine.make("PybulletEngine", rate=200, gui=False, egl=False, process=eagerx.process.ENVIRONMENT)
eval_env = EvaluateEnv(env, engine, episode_timeout=40, render="pybullet")
eval_env.render("human")
path = "last_policy"
# path = "logs/Quadruped_1/rl_model_6000_steps"
model = TQC.load(path)
mean_reward, std = evaluate_policy(model, Flatten(eval_env), n_eval_episodes=5)
print(f"Mean reward = {mean_reward:.2f} +/- {std}")


[WARN] [1653176554.888343]: Adding state "quadruped/image/pos" to simulation node "quadruped/image" can potentially make the agnostic environment with object "quadruped" engine-specific. Check the spec of "Quadruped" under engine implementation "PybulletEngine" for more info.
[WARN] [1653176554.889516]: Adding state "quadruped/image/orientation" to simulation node "quadruped/image" can potentially make the agnostic environment with object "quadruped" engine-specific. Check the spec of "Quadruped" under engine implementation "PybulletEngine" for more info.


[INFO] [1653176554.942069]: Node "/QuadEnv_eval/env/supervisor" initialized.
argv[0]=
argv[0]=
[INFO] [1653176555.101325]: Node "/QuadEnv_eval/engine" initialized.
[INFO] [1653176555.289906]: Node "/QuadEnv_eval/environment" initialized.
[INFO] [1653176555.408301]: Node "/QuadEnv_eval/cartesian_control" initialized.
TROT
[INFO] [1653176555.476807]: Node "/QuadEnv_eval/cpg" initialized.
[INFO] [1653176555.695379]: Adding object "quadruped" of type "Quadruped" to the simulator.
/home/bas/eagerx_dev/eagerx_tutorials/eagerx_tutorials/quadruped/go1/go1_description/urdf/go1.urdf


/home/bas/.cache/pypoetry/virtualenvs/eagerx-tutorials-uyJhHlY9-py3.8/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


[INFO] [1653176555.946218]: Node "/QuadEnv_eval/quadruped/joint_position" initialized.
[INFO] [1653176555.966999]: Node "/QuadEnv_eval/quadruped/joint_velocity" initialized.
[INFO] [1653176555.988785]: Node "/QuadEnv_eval/quadruped/force_torque" initialized.
[INFO] [1653176556.005551]: Node "/QuadEnv_eval/quadruped/orientation" initialized.
[INFO] [1653176556.023348]: Node "/QuadEnv_eval/quadruped/position" initialized.
[INFO] [1653176556.040077]: Node "/QuadEnv_eval/quadruped/velocity" initialized.
[INFO] [1653176556.057581]: Node "/QuadEnv_eval/quadruped/joint_control" initialized.
[INFO] [1653176556.083647]: Node "/QuadEnv_eval/quadruped/image" initialized.
[INFO] [1653176557.411853]: Nodes initialized.
[INFO] [1653176557.694686]: Pipelines initialized.
